# UNIDADE 3 – Teste Estrutural de Software (Fluxo de Dados – Def–Use)

Este notebook implementa:
- Código sob teste em Python
- Mapeamento de associações Def–Use (DU)
- Casos de teste com asserções automatizadas
- Relatório de cobertura dos DUs por caso de teste

**Repositório com os materiais:** https://github.com/vinifcastro/Testes-de-Software/tree/main/UA%203%20-%20Teste%20Estrutural%20de%20Software

## Código sob teste

In [ ]:
def processar(nums):
    """
    Processa uma lista de inteiros até encontrar 0 (sentinela) ou o fim.
    Calcula soma dos positivos, contagem de positivos/negativos, máximo,
    média dos positivos e um fatorial truncado em 5! baseado na contagem de positivos.
    Retorna também o índice onde o laço parou.
    """
    total = 0            # D(t0)
    qtd_pos = 0          # D(cp0)
    qtd_neg = 0          # D(cn0)
    max_val = None       # D(m0)
    i = 0                # D(i0)

    # Laço 1: percorre até achar 0 (sentinela) ou o fim
    while i < len(nums):                     # U(i) (p-uso)
        x = nums[i]                          # D(x0)
        if x == 0:                           # U(x) (p-uso)
            break
        if x > 0:                            # U(x) (p-uso)
            total += x                       # U(total), U(x) (c-uso) -> D(t+)
            qtd_pos += 1                     # U(cp), D(cp+)
            if max_val is None or x > max_val:   # U(max_val) (p-uso), U(x) (p-uso)
                max_val = x                  # D(m+)
        else:
            qtd_neg += 1                     # U(cn), D(cn+)
        i += 1                               # U(i), D(i+)

    # Laço 2: fatorial truncado em 5!
    fat = 1
    for k in range(1, min(qtd_pos, 5) + 1):  # U(qtd_pos) (p-uso)
        fat *= k

    media = total / qtd_pos if qtd_pos > 0 else 0  # U(total) (c-uso), U(qtd_pos) (p-uso)
    return {
        "total": total,
        "qtd_pos": qtd_pos,
        "qtd_neg": qtd_neg,
        "max": max_val,
        "media": media,
        "fat": fat,
        "stopped_at": i,
    }

## Associações Def–Use consideradas

### Mapeamento Def–Use (DU) considerado

- **i**
  - DU1: D(i0) → U(i) em `while i < len(nums)`
  - DU2: D(i+) → U(i) na iteração seguinte do `while`

- **x**
  - DU3: D(x0) → U(x) em `x == 0` (p-uso)
  - DU4: D(x0) → U(x) em `x > 0` (p-uso)
  - DU5: D(x0) → U(x) em `total += x` (c-uso)

- **total**
  - DU6: D(t0=0) → U(total) no cálculo da `media` quando `qtd_pos == 0`
  - DU7: D(t0) → U(total) na primeira ocorrência de `total += x`
  - DU8: D(t+) (última do laço) → U(total) no cálculo da `media` (`qtd_pos > 0`)

- **qtd_pos**
  - DU9:  D(cp0=0) → U(qtd_pos) no predicado `qtd_pos > 0` (ramo falso)
  - DU10: D(cp0=0) → U(qtd_pos) em `min(qtd_pos,5)` (laço 2 não executa)
  - DU11: D(cp+)   → U(qtd_pos) em `min(qtd_pos,5)` (laço 2 executa)
  - DU12: D(cp+)   → U(qtd_pos) no predicado `qtd_pos > 0` (ramo verdadeiro)

- **qtd_neg**
  - DU13: D(cn0=0) → uso no retorno
  - DU14: D(cn+)   → uso no retorno

- **max_val**
  - DU15: D(m0=None) → U(max_val) em `max_val is None`
  - DU16: D(m+) → U(max_val) em `x > max_val`
  - DU17: D(m+) → uso no retorno

## Casos de Teste (definição)

In [ ]:
# Definição dos casos de teste e DUs esperados por CT
test_cases = [
  {
    "id": "CT1",
    "entrada": [],
    "esperado": {
      "total": 0,
      "qtd_pos": 0,
      "qtd_neg": 0,
      "max": None,
      "media": 0,
      "fat": 1,
      "stopped_at": 0
    }
  },
  {
    "id": "CT2",
    "entrada": [3, 2, 1],
    "esperado": {
      "total": 6,
      "qtd_pos": 3,
      "qtd_neg": 0,
      "max": 3,
      "media": 2.0,
      "fat": 6,
      "stopped_at": 3
    }
  },
  {
    "id": "CT3",
    "entrada": [-5, -2, 0],
    "esperado": {
      "total": 0,
      "qtd_pos": 0,
      "qtd_neg": 2,
      "max": None,
      "media": 0,
      "fat": 1,
      "stopped_at": 2
    }
  },
  {
    "id": "CT4",
    "entrada": [-1, 4, -2, 6],
    "esperado": {
      "total": 10,
      "qtd_pos": 2,
      "qtd_neg": 2,
      "max": 6,
      "media": 5.0,
      "fat": 2,
      "stopped_at": 4
    }
  },
  {
    "id": "CT5",
    "entrada": [0],
    "esperado": {
      "total": 0,
      "qtd_pos": 0,
      "qtd_neg": 0,
      "max": None,
      "media": 0,
      "fat": 1,
      "stopped_at": 0
    }
  },
  {
    "id": "CT6",
    "entrada": [1, 2, 3, 4, 5, 6],
    "esperado": {
      "total": 21,
      "qtd_pos": 6,
      "qtd_neg": 0,
      "max": 6,
      "media": 3.5,
      "fat": 120,
      "stopped_at": 6
    }
  }
]
ct_to_du = {
  "CT1": ["DU1","DU6","DU9","DU10","DU13"],
  "CT2": ["DU1","DU2","DU3","DU4","DU5","DU7","DU8","DU11","DU12","DU15","DU16","DU13","DU17"],
  "CT3": ["DU1","DU2","DU3","DU9","DU10","DU13","DU14"],
  "CT4": ["DU1","DU2","DU3","DU4","DU5","DU7","DU8","DU11","DU12","DU14","DU15","DU16","DU17"],
  "CT5": ["DU1","DU3","DU6","DU9","DU10","DU13"],
  "CT6": ["DU2","DU4","DU5","DU7","DU8","DU11","DU12","DU15","DU16","DU17"]
}
all_du = ["DU1","DU2","DU3","DU4","DU5","DU6","DU7","DU8","DU9","DU10","DU11","DU12","DU13","DU14","DU15","DU16","DU17"]

# Exibe resumo rápido
(len(test_cases), list(ct_to_du.keys())[:6], len(all_du))

## Execução dos testes e verificação de resultados

In [ ]:
import math
from copy import deepcopy
import pandas as pd

exec_results = []

for tc in test_cases:
    entrada = deepcopy(tc["entrada"])
    got = processar(entrada)
    exp = tc["esperado"]
    # comparações com tolerância para floats
    same = True
    diffs = {}
    for k in exp.keys():
        gv = got[k]
        ev = exp[k]
        if isinstance(ev, float):
            ok = math.isclose(gv, ev, rel_tol=1e-9, abs_tol=1e-9)
        else:
            ok = (gv == ev)
        if not ok:
            same = False
            diffs[k] = (ev, gv)
    exec_results.append({
        "CT": tc["id"],
        "Entrada": str(entrada),
        "Passou": same,
        "Divergências": diffs if diffs else "-",
        **{k: got[k] for k in ["total","qtd_pos","qtd_neg","max","media","fat","stopped_at"]}
    })

df_results = pd.DataFrame(exec_results)
df_results

## Cobertura de DUs

In [ ]:
import pandas as pd

# Conjunto de DUs cobertos (união de todos os CTs)
covered = set()
for ct, dus in ct_to_du.items():
    covered.update(dus)

coverage_pct = 100.0 * len(covered) / len(all_du)

# DataFrame de cobertura por DU e por CT (marcação simples)
rows = []
for du in all_du:
    row = {"DU": du, "Coberto": du in covered}
    for ct in ct_to_du.keys():
        row[ct] = ("✔️" if du in ct_to_du[ct] else "—")
    rows.append(row)
df_cov = pd.DataFrame(rows).set_index("DU")

print(f"Total de DUs: {len(all_du)}")
print(f"Cobertos: {len(covered)}")
print(f"Cobertura: {coverage_pct:.2f}%")

df_cov

## Conclusões

- Os 17 elementos Def–Use mapeados foram cobertos pelos conjuntos de testes definidos.
- A suíte contempla:
  - Saída por sentinela (`x == 0`).
  - Ausência de positivos (média zero, laço 2 não executa).
  - Somente positivos (média > 0, laço 2 executa).
  - Atualização de máximo com ramo verdadeiro e falso.
  - Presença de negativos e incremento de `qtd_neg`.

> Nota: a cobertura aqui é calculada a partir do mapeamento DU ↔ CT planejado.

**Repositório:** https://github.com/vinifcastro/Testes-de-Software/tree/main/UA%203%20-%20Teste%20Estrutural%20de%20Software